In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score


C:\Users\Surya\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


### Reading the final train data


In [2]:
final=pd.read_csv('final.csv')

### Reading the final test data


In [3]:
#final_test=pd.read_csv('final_test.csv')

### Changing the type as category for categorical features

In [4]:
final["city"] = final["city"].astype("category")
final["registered_via"] = final["registered_via"].astype("category")
final["registration_init_time_year"] = final["registration_init_time_year"].astype("category")
final["registration_init_time_month"] = final["registration_init_time_month"].astype("category")
final["registration_init_time_day"] = final["registration_init_time_day"].astype("category")
final["payment_method_id"] = final["payment_method_id"].astype("category")
final["payment_plan_days"] = final["payment_plan_days"].astype("category")
final["is_cancel"] = final["is_cancel"].astype("category")
final["is_auto_renew"] = final["is_auto_renew"].astype("category")
final["transaction_date_year"] = final["transaction_date_year"].astype("category")
final["transaction_date_month"] = final["transaction_date_month"].astype("category")
final["transaction_date_day"] = final["transaction_date_day"].astype("category")

In [5]:
#final_test["city"] = final_test["city"].astype("category")
#final_test["registered_via"] = final_test["registered_via"].astype("category")
#final_test["registration_init_time_year"] = final_test["registration_init_time_year"].astype("category")
#final_test["registration_init_time_month"] = final_test["registration_init_time_month"].astype("category")
#final_test["registration_init_time_day"] = final_test["registration_init_time_day"].astype("category")
#final_test["payment_method_id"] = final_test["payment_method_id"].astype("category")
#final_test["payment_plan_days"] = final_test["payment_plan_days"].astype("category")
#final_test["is_cancel"] = final_test["is_cancel"].astype("category")
#final_test["is_auto_renew"] = final_test["is_auto_renew"].astype("category")
#final_test["transaction_date_year"] = final_test["transaction_date_year"].astype("category")
#final_test["transaction_date_month"] = final_test["transaction_date_month"].astype("category")
#final_test["transaction_date_day"] = final_test["transaction_date_day"].astype("category")

### Test train split

In [4]:
Y_train = final['is_churn'].values
final.drop(['is_churn','msno'], axis=1, inplace=True)

In [5]:
X_train = final
X_train.head(1)

,city,registered_via,registration_init_time_year,registration_init_time_month,registration_init_time_day,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_cancel,...,transaction_date_month,transaction_date_day,logs_count,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,5.0,3.0,2013,12,23,38,30,149,149,0,...,2,28,11.0,186.0,23.0,13.0,10.0,318.0,348.0,80598.557


In [6]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, stratify=Y_train)
X_train, X_cv, Y_train, Y_cv = train_test_split(X_train, Y_train, test_size=0.2, stratify=Y_train)

In [7]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)
print(X_cv.shape, Y_cv.shape)

(482900, 29) (482900,)
(150907, 29) (150907,)
(120725, 29) (120725,)


### Applying StandardScaler on numerical features

In [10]:
scaler = StandardScaler()

In [11]:
X_train[['plan_list_price', 'actual_amount_paid','trans_count','transaction_span','total_list_price','total_amount_paid','is_cancel_sum','difference_in_price_paid','amount_paid_perday','logs_count','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']] = scaler.fit_transform(X_train[['plan_list_price', 'actual_amount_paid','trans_count','transaction_span','total_list_price','total_amount_paid','is_cancel_sum','difference_in_price_paid','amount_paid_perday','logs_count','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']])

In [12]:
X_test[['plan_list_price', 'actual_amount_paid','trans_count','transaction_span','total_list_price','total_amount_paid','is_cancel_sum','difference_in_price_paid','amount_paid_perday','logs_count','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']] = scaler.transform(X_test[['plan_list_price', 'actual_amount_paid','trans_count','transaction_span','total_list_price','total_amount_paid','is_cancel_sum','difference_in_price_paid','amount_paid_perday','logs_count','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']])

In [13]:
X_cv[['plan_list_price', 'actual_amount_paid','trans_count','transaction_span','total_list_price','total_amount_paid','is_cancel_sum','difference_in_price_paid','amount_paid_perday','logs_count','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']] = scaler.transform(X_cv[['plan_list_price', 'actual_amount_paid','trans_count','transaction_span','total_list_price','total_amount_paid','is_cancel_sum','difference_in_price_paid','amount_paid_perday','logs_count','num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']])

### Final model

In [24]:
def final_model(X,Y):
    """This function is used to get the input X,Y and it gives the log loss and accuracy score based on LGBMClassifier."""
    
    x_cfl=LGBMClassifier(objective ='binary', eval_metric = 'logloss',class_weight='balanced',subsample=0.3,num_leaves =1500,
                    n_estimators=1000,max_depth=5,learning_rate=0.2,colsample_bytree=0.5,feature_fraction=0.2)
    x_cfl.fit(X,Y)
    c_cfl=CalibratedClassifierCV(x_cfl,method='sigmoid')
    c_cfl.fit(X,Y)
    
    predict_y = c_cfl.predict_proba(X)
    print ('log loss is',log_loss(Y, predict_y))

In [25]:
train_metric = final_model(X_train,Y_train)

log loss is 0.08499339948285352


In [26]:
cv_metric = final_model(X_cv,Y_cv)

log loss is 0.0533228009097414


In [27]:
test_metric = final_model(X_test,Y_test)

log loss is 0.05831275557349452


Changing categorical feature type as category gives better result than the LGM model in the "Data modelling and ML Models notebook"

## Summary

1. Imported all the data's and joined into a single dataframe based on msno.

2. With EDA found out the features to be used and not to be used, created new features based on the given features.

3. Seperated features into Categorical and Numerical features

4. For Categorical features did one hot encoding and for numerical features did standard scalar.

5. Tried Logistic regression ,SVM and LightGBM models, Light GBM model with class balance gives the result.

6. In the final notebook implemeted the LightGBM model for prediction.